In [2]:
import os

In [3]:
os.chdir('/Users/anshuljethani/Desktop/NLPproject/Text-Summarizer--NLP-Project')

In [6]:
pwd

'/Users/anshuljethani/Desktop/NLPproject/Text-Summarizer--NLP-Project'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [10]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories,get_size

ModuleNotFoundError: No module named 'textSummarizer'

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=5, warmup_steps=500,
            per_device_train_batch_size=8, per_device_eval_batch_size=8,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=100, save_steps=1e6,
            gradient_accumulation_steps=8
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [13]:
from textSummarizer.logging import logger

In [15]:
from textSummarizer.config.configuration import ConfigurationManager

In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    logger.exception(e)
    raise e

[2024-09-13 17:02:07,007: ERROR: 2115163532: [Errno 2] No such file or directory: 'config/config.yaml']
Traceback (most recent call last):
  File "/var/folders/8p/xwh1nn8d0773ym63vjph24km0000gn/T/ipykernel_17295/2115163532.py", line 2, in <module>
    config = ConfigurationManager()
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/anshuljethani/Desktop/NLPproject/Text-Summarizer--NLP-Project/src/textSummarizer/config/configuration.py", line 13, in __init__
    self.config = read_yaml(config_filepath)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/anshuljethani/Desktop/NLPproject/Text-Summarizer--NLP-Project/venv/lib/python3.12/site-packages/ensure/main.py", line 872, in __call__
    return_val = self.f(*args, **kwargs)
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/anshuljethani/Desktop/NLPproject/Text-Summarizer--NLP-Project/src/textSummarizer/utils/common.py", line 21, in read_yaml
    raise Exception
    ^^^^^^^
  File "/Users/anshuljethani/Desktop/NLPproject

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'